# Homework 1
Pedro Augusto Januzzi Guerra

I'm auditing the course, so I'll only be solving questions 1 and 3.

## Question 1

I start by coding a function that computes the root of a quadratic equation using the Bhaskara's formula.

In [ ]:
function bhaskara(a,b,c)
    Δ = b^2 - 4*a*c;
    roots = Vector{Float64}(undef, 2);
    
    roots[1] = (1/2*a)*(-b + sqrt(Δ));
    roots[2] = (1/2*a)*(-b - sqrt(Δ));

    return roots
end

Compute larger root of $ax^2 + bx + c$ for $a=1$, $b=100,000$, and $c = 10^n$ for $n=-1,-2,\cdots,-8$.

In [ ]:
a = 1
b = 1e5
n = -1:-1:-8  
solution = [bhaskara(a, b, 10.0^val) for val in n]

Now I will use the alternative method:

In [ ]:
function alt_method(a,b,c)
    Δ = b^2 - 4*a*c;
    roots = Vector{Float64}(undef, 2);

    q = -0.5*(b + sign(b)*sqrt(Δ));

    roots[2] = q/a;
    roots[1] = c/q;

    return roots
end

In [ ]:
solution_alt = [alt_method(a, b, 10.0^val) for val in n]

As the question states, the two methods should yield the same answer. However, due to the rounding error of the computer arithmetic, the results for the larger root are not equal.

Question: how close are the answers for different values of n?

To answer the question, I compute the difference between the larger roots of both methods for each $n = -1,\cdots,-8$:

In [ ]:
diff = [solution[i][1] - solution_alt[i][1] for i in eachindex(n)]

In [ ]:
using PrettyTables

n_values = ["n=-1", "n=-2", "n=-3", "n=-4", "n=-5", "n=-6", "n=-7", "n=-8"]
table_data = hcat(n_values, string.(diff))

# Print the table with a title
pretty_table(table_data, header=["n Value", "Difference"], title="Difference between larger roots of both methods")


As mentioned earlier, while the answers are not identical computationally, they are extremely close, with differences appearing only in the 12th or 13th decimal place

---
## Question 3

Initialize a struct and assign some parameters to it:

In [80]:
using Parameters

# Now you can redefine it with @with_kw
@with_kw mutable struct NeoclassicalGrowthModel
    β::Float64 = 0.98;  # discount factor
    α::Float64 = 0.4;   # capital share
    δ::Float64 = 0.1;   # depreciation rate
    n::Int = 1;         # number of grid points
    kgrid::Vector{Float64} = zeros(1)  # capital grid vector
    A::Float64 = 1.0; # tfp level
end


NeoclassicalGrowthModel

Initialize an instance:

In [81]:
NGM = NeoclassicalGrowthModel()

NeoclassicalGrowthModel
  β: Float64 0.98
  α: Float64 0.4
  δ: Float64 0.1
  n: Int64 1
  kgrid: Array{Float64}((1,)) [0.0]
  A: Float64 1.0


Create a grid for capital around the steady state level:

In [49]:
@unpack α,β,δ,A = NGM;

# Calculate the steady state capital level for A=1:
k_ss = (1/(A*α)*((1/β)+δ-1))^(1/(α-1));

# Create a linearly spaced grid around the steady state value:
lb = 0.6*k_ss; # 40% decrease of k_ss 
ub = 1.4*k_ss; # 40% increase of k_ss
n = 11; # number of grid points

# Grid: 
kgrid = LinRange(lb,ub,n);

# Include grid and number of points in the struct
NGM.kgrid = kgrid;
NGM.n = n;

Create a function to compute utility:

In [50]:
u = c -> log(c);

In [51]:
using LinearAlgebra

Create a function for VFI where the stopping rule is based on value function convergence (not used here, but might be useful in the future):

In [ ]:
function vfi1(NGM::NeoclassicalGrowthModel,Vguess::Vector{Float64},tol::Float64) 
    @unpack α,β,δ,n,kgrid = NGM;

    # Initialize values for the interation 
    dif = 1;
    iter = 0;
    Vold = Vguess;
    Vnew = Vold;
    k′ = Vector{Float64}(undef,n); # policy function for capital
    
    # vfi 
    while dif>tol
        Vold = copy(Vnew);
        for i = 1:n 
            vec = Vector{Float64}(undef,n);
            for j = 1:n # loop on k′
                # compute consumption 
                c = kgrid[i]^α + (1-δ)*kgrid[i] - kgrid[j]; # note that I'm assuming k′ lies on the same grid as k
                
                # compute utility 
                if c > 0
                    util = u(c);
                else
                    util = -Inf;
                end

                # vector to evaluate and choose optimal k′
                vec[j] = util + β*Vold[j];
            end

            # compute optimal choice for k′
            Vnew[i] = maximum(vec);
            idx = findfirst(v -> v == Vnew[i], vec);
            k′[i] = kgrid[idx];
        end

        # Stopping rule based on value function convergence:
        dif = norm(Vnew-Vold,Inf);
        iter += 1;

        # Print iteration and difference between Vnew and Vold
        println("Iteration $iter: Difference between Vnew and Vold = ", dif)
    end

    return Vnew, k′, iter
end

Create a function for VFI where the stopping rule is based on policy function convergence:

In [ ]:
function vfi2(NGM::NeoclassicalGrowthModel, Vguess::Vector{Float64}, tol::Float64)::Tuple{Vector{Float64}, Vector{Float64}, Int64}
    @unpack α, β, δ, n, kgrid = NGM;

    # Initialize values for the iteration 
    dif = 1;
    iter = 0;
    Vold = Vguess;
    Vnew = Vold;
    k′ = Vector{Float64}(undef, n);  # policy function for capital
    k′old = copy(k′);  # initialize previous policy function for convergence check
    
    # vfi 
    while dif > tol
        k′old = copy(k′)  # Update previous policy function
        Vold = copy(Vnew);
    
        for i = 1:n 
            vec = Vector{Float64}(undef, n);
            
            for j = 1:n  # loop on k′
                # compute consumption 
                c = kgrid[i]^α + (1 - δ) * kgrid[i] - kgrid[j];  # assuming k′ lies on the same grid as k
                
                # compute utility 
                if c > 0
                    util = u(c);
                else
                    util = -Inf;
                end

                # vector to evaluate and choose optimal k′
                vec[j] = util + β * Vold[j];
            end

            # compute optimal choice for k′
            Vnew[i] = maximum(vec);
            idx = findfirst(v -> v == Vnew[i], vec);
            k′[i] = kgrid[idx];
        end

        # Stopping rule based on policy function convergence:
        dif = norm(k′ - k′old, Inf);  # Convergence based on the change in the policy function        
        iter += 1;

        # Print iteration and difference between k′ and k′old
        println("Iteration $iter: Policy function difference = ", dif)
    end

    return Vnew, k′, iter
end


### Item (a)

I will solve the VFI for different grid sizes: $n=11$, $n=101$, and $n=1001$.

In [ ]:
n_values = [11, 101, 1001]; # possible grid sizes
V = Vector{Vector{Float64}}(undef, length(n_values));  # create an array to store value function for all cases
k′ = Vector{Vector{Float64}}(undef, length(n_values)); # create an array to store policy function for all cases
iter = Vector{Float64}(undef,length(n_values)); # create an array to store the number of iterations
grid_aux = Vector{Vector{Float64}}(undef, length(n_values)); # create an array to store the kgrids

tol = 1e-8; # tolerance

for i in eachindex(n_values)
    V[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    k′[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    grid_aux[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    
    n = n_values[i]; # number of grid points
    kgrid = LinRange(lb,ub,n); # capital grid based on number of grid points
    NGM.kgrid = kgrid; # update kgrid in the struct
    NGM.n = n; # update n in the struct

    grid_aux[i] = kgrid;
    
    Vguess = zeros(n);
    
    V[i], k′[i], iter[i] = vfi2(NGM, Vguess, tol);

end


#### Subitem (i)

Let's compute the number of iterations it took to converge for all grid sizes considered:

In [ ]:
n_title = ["n=11", "n=101", "n=1001"]
table_data = hcat(n_title, string.(iter))

# Print the table with a title
pretty_table(table_data, header=["Grid points", "Number of iterations"], title="Number of iterations it took for VFI to stop")

#### Subitem (ii)

Plot value and policy function side by side for n = 11:

In [ ]:
using Plots 
# create two plots side by side
plot1 = plot(grid_aux[1,], V[1,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[1,], k′[1,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

Plot value and policy function side by side for n = 101:

In [ ]:
# create two plots side by side
plot1 = plot(grid_aux[2,], V[2,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[2,], k′[2,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

Plot value and policy function side by side for n = 1001:

In [ ]:
# create two plots side by side
plot1 = plot(grid_aux[3,], V[3,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[3,], k′[3,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

As shown in the figures above, the value function is strictly increasing and strictly concave, which aligns with expectations given the utility and production functions. These functions satisfy the assumptions outlined in Chapter 4 of SLP, ensuring that the theorems discussed in the book—implying strict monotonicity and concavity—hold.

Regarding the policy function, we expected it to be strictly increasing. However, as observed (particularly for the cases with $n=11$ and $n=101$) the policy function is not strictly increasing but simply increasing. This behavior is a direct consequence of our decision to use a coarse grid and allow $k^\prime$ to lie on the same grid as $k$

### Item (b)

I will solve item (a) again but with different tolerances. First, I use $10^{-6}$ as tolerance.

In [ ]:
tol = 1e-6;

n_values = [11, 101, 1001]; # possible grid sizes
V = Vector{Vector{Float64}}(undef, length(n_values));  # create an array to store value function for all cases
k′ = Vector{Vector{Float64}}(undef, length(n_values)); # create an array to store policy function for all cases
iter = Vector{Float64}(undef,length(n_values)); # create an array to store the number of iterations
grid_aux = Vector{Vector{Float64}}(undef, length(n_values)); # create an array to store the kgrids



for i in eachindex(n_values)
    V[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    k′[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    grid_aux[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    
    n = n_values[i]; # number of grid points
    kgrid = LinRange(lb,ub,n); # capital grid based on number of grid points
    NGM.kgrid = kgrid; # update kgrid in the struct
    NGM.n = n; # update n in the struct

    grid_aux[i] = kgrid;
    
    Vguess = zeros(n);
    
    V[i], k′[i], iter[i] = vfi2(NGM, Vguess, tol);

end

Number of iterations:

In [ ]:
n_title = ["n=11", "n=101", "n=1001"]
table_data = hcat(n_title, string.(iter))

# Print the table with a title
pretty_table(table_data, header=["Grid points", "Number of iterations"], title="Number of iterations it took for VFI to stop")

Graphs for $n = 11$:

In [ ]:
# create two plots side by side
plot1 = plot(grid_aux[1,], V[1,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[1,], k′[1,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

Graphs for $n=101$:

In [ ]:
# create two plots side by side
plot1 = plot(grid_aux[2,], V[2,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[2,], k′[2,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

Graphs for $n=1001$:

In [ ]:
# create two plots side by side
plot1 = plot(grid_aux[3,], V[3,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[3,], k′[3,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

Now I do the same but using a tolerance of $10^{-5}$.

In [ ]:
tol = 1e-5;

n_values = [11, 101, 1001]; # possible grid sizes
V = Vector{Vector{Float64}}(undef, length(n_values));  # create an array to store value function for all cases
k′ = Vector{Vector{Float64}}(undef, length(n_values)); # create an array to store policy function for all cases
iter = Vector{Float64}(undef,length(n_values)); # create an array to store the number of iterations
grid_aux = Vector{Vector{Float64}}(undef, length(n_values)); # create an array to store the kgrids



for i in eachindex(n_values)
    V[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    k′[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    grid_aux[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    
    n = n_values[i]; # number of grid points
    kgrid = LinRange(lb,ub,n); # capital grid based on number of grid points
    NGM.kgrid = kgrid; # update kgrid in the struct
    NGM.n = n; # update n in the struct

    grid_aux[i] = kgrid;
    
    Vguess = zeros(n);
    
    V[i], k′[i], iter[i] = vfi2(NGM, Vguess, tol);

end

In [ ]:
n_title = ["n=11", "n=101", "n=1001"]
table_data = hcat(n_title, string.(iter))

# Print the table with a title
pretty_table(table_data, header=["Grid points", "Number of iterations"], title="Number of iterations it took for VFI to stop")

In [ ]:
# create two plots side by side
plot1 = plot(grid_aux[1,], V[1,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[1,], k′[1,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

In [ ]:
# create two plots side by side
plot1 = plot(grid_aux[2,], V[2,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[2,], k′[2,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

In [ ]:
# create two plots side by side
plot1 = plot(grid_aux[3,], V[3,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[3,], k′[3,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

Summary: The number of iterations has remained unchanged, and the graphs appear similar to those from part (a). While this might initially seem unusual, the convergence of the policy function clarifies the situation. As the difference between the policy functions from two consecutive iterations converges precisely to zero in all cases above, adjusting the tolerance from $10^{-8}$ to $10^{-6}$ or $10^{-5}$ does not affect the results.

### Item (c)

First, I will repeat part (a) considering full depreciation ($\delta=1$).

In [ ]:
tol = 1e-8;
δ = 1;
NGM.δ = δ;

# update grid choices
k_ss = (1/(A*α)*((1/β)+δ-1))^(1/(α-1));
lb = 0.6*k_ss; # 40% decrease of k_ss 
ub = 1.4*k_ss; # 40% increase of k_ss

n_values = [11, 101, 1001]; # possible grid sizes
V = Vector{Vector{Float64}}(undef, length(n_values));  # create an array to store value function for all cases
k′ = Vector{Vector{Float64}}(undef, length(n_values)); # create an array to store policy function for all cases
iter = Vector{Float64}(undef,length(n_values)); # create an array to store the number of iterations
grid_aux = Vector{Vector{Float64}}(undef, length(n_values)); # create an array to store the kgrids



for i in eachindex(n_values)
    V[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    k′[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    grid_aux[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    
    n = n_values[i]; # number of grid points
    kgrid = LinRange(lb,ub,n); # capital grid based on number of grid points
    NGM.kgrid = kgrid; # update kgrid in the struct
    NGM.n = n; # update n in the struct

    grid_aux[i] = kgrid;
    
    Vguess = zeros(n);
    
    V[i], k′[i], iter[i] = vfi2(NGM, Vguess, tol);

end

Number of iterations:

In [ ]:
n_title = ["n=11", "n=101", "n=1001"]
table_data = hcat(n_title, string.(iter))

# Print the table with a title
pretty_table(table_data, header=["Grid points", "Number of iterations"], title="Number of iterations it took for VFI to stop")

Plot graphs for $n=11$, $n=101$, and $n=1001$, respectively:

In [ ]:
# create two plots side by side
plot1 = plot(grid_aux[1,], V[1,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[1,], k′[1,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

In [ ]:
# create two plots side by side
plot1 = plot(grid_aux[2,], V[2,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[2,], k′[2,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

In [ ]:
# create two plots side by side
plot1 = plot(grid_aux[3,], V[3,], title="Value Function", label=false, xlabel="k", ylabel="V")
plot2 = plot(grid_aux[3,], k′[3,], title="Policy Function", label=false, xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)

#### Subitem (i)

First, I will compute the true solution:

In [75]:
# Get a0 and a1
a1 = α/(1-α*β);
a0 = (1/(1-β))*(1/(1-α*β))*(log(A) - log(1/(1-α*β)) + β*α*log(β) + β*α*log(α/(1-α*β))); 

n_values = [11, 101, 1001]; # possible grid sizes
Vtrue = Vector{Vector{Float64}}(undef, length(n_values));
k′_true = Vector{Vector{Float64}}(undef, length(n_values));
grid_aux = Vector{Vector{Float64}}(undef, length(n_values)); 

for i in eachindex(n_values)
    Vtrue[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    k′_true[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    grid_aux[i] = Vector{Float64}(undef, n_values[i]); # initialize inner vector
    
    n = n_values[i]; # number of grid points
    kgrid = LinRange(lb,ub,n);
    grid_aux[i] = kgrid;

    for j = 1:n 
        Vtrue[i][j] = a0 + a1*log(kgrid[j]);
        k′_true[i][j] = β*a1*A*kgrid[j]^α/(1+β*a1);
    end
    
end

Plot the analytical and numerical solutions:

In [ ]:
plot1 = plot(grid_aux[3,], V[3,], title="Value Function", label="Numerical Solution", xlabel="k", ylabel="V")
plot2 = plot(grid_aux[3,], Vtrue[3,], title="Value Function", label="Analytical Solution", xlabel="k", ylabel="k'")

# display the two plots side by side
plot(plot1, plot2, layout=(1, 2),size=(800, 400),margin = 5Plots.mm)


The difference between the value function values in the numerical and analytical solutions is quite evident on the y-axis, making it clear that plotting them on the same graph wouldn't be meaningful.

However, for the policy function, the analytical and numerical solutions are very similar, as shown in the next graph...

In [ ]:
plot(grid_aux[3,], k′[3,], title="Policy Function", label="Numerical Solution", xlabel="k", ylabel="k'")
plot!(grid_aux[3,], k′_true[3,], title="Policy Function", label="Analytical Solution", xlabel="k", ylabel="k'")

...although they are not identical. Zooming in reveals the differences more clearly:

In [ ]:
plot(grid_aux[3,], k′[3,], title="Policy Function", label="Numerical Solution", xlabel="k", ylabel="k'",xlims=(0.18,0.185),margin = 5Plots.mm)
plot!(grid_aux[3,], k′_true[3,], title="Policy Function", label="Analytical Solution", xlabel="k", ylabel="k'",ylims=(0.1972,0.1999))

#### Subitem (ii)

In [ ]:

n_values = [11, 101, 1001]
iterations = [
    9  9  9 7;
    36  36  36 11;
    72  72  72 16
] # I decided to write it manually to avoid coming back and change several pieces of the code


# Headers
header = ["n", "Item (a)", "Item (b) - tol 10^{-6}", "Item (b) - tol 10^{-5}", "Item (c)"]

# Add n values as the first column of iterations data
table_data = hcat(n_values, iterations)

# Print the table with a title
pretty_table(table_data, header=header, title="Number of iterations")

